# *Valor Patrimonial de Imóveis em SP*

---

Luiz Guilherme Gomes Fregona

Abril, 2022

## 1.   Introdução

*Uma reserva de valor tem como principal característica a manutenção do poder de compra do investidor ao decorrer do tempo. Conforme muitos analistas e consultores de mercado, o ouro, terras e imóveis são ótimas reservas de valor. De modo a averiguar se tal hipótese se mantém verdade no estado de São Paulo, o presente projeto irá comparar o valor venal de imóveis na região com o valor total da inflação brasileira de 1995 até o presente momento, além de produzir um modelo estatístico estimando seus valores no futuro. Os dados são open source e podem ser encontrados no seguinte website: geosampa.prefeitura.sp.gov.br/PaginasPublicas/_SBC.aspx*

## 2.   Objetivos

*Confirmar a hipótese "Imóveis protegem contra a inflação" e prever o valor venal futuro de imóveis em diferentes regiões de São Paulo.*

## 3.   Metodologia

*A metodologia utilizada nesse trabalho engloba todas as etapas do framework CRISP-DM.*

## 4. Data Scraping

### 4.1 Definir arquivos de controle do fluxo ETL (logfile) e banco de dados (targetfile)

In [1]:
logfile    = "logfile.txt"      # Todos os eventos performados e seus horários serão armazenados nesse arquivo
targetfile = "dados_iptu.csv"   # Arquivo onde todos os arquivos .csv serão armazenas após serem transformados

### 4.2 Importar todas as bibliotecas

In [2]:
#Import libraries
try:
  import sys  
except ImportError:
  prosseguir = input("Você não possui o pacote sys! Você deseja instalá-lo? (s/n)")
  if prosseguir == 's':
    %pip install sys
    import sys 
  else:
    sys.exit("Tente novamente!")

try:
  import requests
except ImportError:
  prosseguir = input("Você não possui o pacote requests! Você deseja instalá-lo? (s/n)")
  if prosseguir == 's':
    %pip install requests
    import requests
  else:
    sys.exit("Tente novamente!")    

try:
  import os
  os.system('apt update')
  os.system('apt install chromium-chromedriver')
except ImportError:
  prosseguir = input("Você não possui o pacote os! Você deseja instalá-lo? (s/n)")
  if prosseguir == 's':
    %pip install os
    import os
  else:
    sys.exit("Tente novamente!")

try:
  import glob
except ImportError:
  prosseguir = input("Você não possui o pacote glob! Você deseja instalá-lo? (s/n)")
  if prosseguir == 's':
    %pip install glob
    import glob
  else:
    sys.exit("Tente novamente!")

try:
  import pandas as pd
except ImportError:
  prosseguir = input("Você não possui o pacote pandas! Você deseja instalá-lo? (s/n)")
  if prosseguir == 's':
    %pip install pandas
    import pandas as pd
  else:
    sys.exit("Tente novamente!")

try:
  import zipfile
except ImportError:
  prosseguir = input("Você não possui o pacote zipfile! Você deseja instalá-lo? (s/n)")
  if prosseguir == 's':
    %pip install zipfile
    import zipfile
  else:
    sys.exit("Tente novamente!")

try:
  from selenium import webdriver 
  from selenium.webdriver.common.by import By
  from selenium.webdriver.support.select import Select
except ImportError:
  prosseguir = input("Você não possui o pacote selenium! Você deseja instalá-lo? (s/n)")
  if prosseguir == 's':
    %pip install selenium
    from selenium import webdriver 
    from selenium.webdriver.common.by import By
    from selenium.webdriver.support.select import Select
  else:
    sys.exit("Tente novamente!")

try:
  from webdriver_manager.chrome import ChromeDriverManager
except ImportError:
  prosseguir = input("Você não possui o pacote webdriver_manager! Você deseja instalá-lo? (s/n)")
  if prosseguir == 's':
    %pip install webdriver_manager
    from webdriver_manager.chrome import ChromeDriverManager
  else:
    sys.exit("Tente novamente!")

try:
  import time
except ImportError:
  prosseguir = input("Você não possui o pacote time! Você deseja instalá-lo? (s/n)")
  if prosseguir == 's':
    %pip install time
    import time
  else:
    sys.exit("Tente novamente!")

### 4.3 Extract - Download dos documentos remotos para diretório local

In [3]:
def download_iptu (ano_iptu, driver):
    iptu = "IPTU_" + str(ano_iptu)
    iptu_click = driver.find_element(By.XPATH, value = '//*[contains(text(), "{}")]'.format(iptu))
    driver.execute_script("arguments[0].click();", iptu_click)

def parafrente_pagina(driver):
    icone_nextpage = driver.find_element(By.XPATH, value="//span[contains(@class, 'next')]") 
    driver.execute_script("arguments[0].click();", icone_nextpage)

def paratras_pagina(driver):
    icone_lastpage = driver.find_element(By.XPATH, value="//span[contains(@class, 'prev')]") 
    driver.execute_script("arguments[0].click();", icone_lastpage)
    
def contagem_documentos(driver):
    n_paginas = 0
    sum_docs = 0
    while True: #Conta quantos documentos existem no total
        docs_porpagina = len(driver.find_elements(By.XPATH, value = '//*[contains(text(), "IPTU_")]'))
        if docs_porpagina == 8:
            sum_docs += docs_porpagina
            n_paginas += 1
            parafrente_pagina(driver)
        else:
            docs_porpagina = len(driver.find_elements(By.XPATH, value = '//*[contains(text(), "IPTU_")]'))
            sum_docs += docs_porpagina
            n_paginas += 1
            break

    for cada_pagina in range(n_paginas): #Volta para a pagina inicial após contagem
        paratras_pagina(driver)
    
    return sum_docs

def caminho_iptu (ano_iptu): #Define o caminho do arquivo baseado em seu nome
    iptu = "IPTU_" + str(ano_iptu) + ".zip"
    caminho = str

    for r,d,f in os.walk("c:\\Users\\User\\Downloads"): #Pode ser alterado para aplicações mais genericas
         for files in f:
              if files == iptu:
                   caminho = os.path.join(r,files)
     
    return caminho

def extrair_zipfile(caminho_arquivo): #Extrair os arquivos .csv, e tranforma-los em um único DataFrame para correções na etapa de transformação:
      with zipfile.ZipFile(caminho_arquivo, 'r') as zip_ref:
           zip_ref.extractall("./csv")

def extrair_csv(caminho_csv):
     df = pd.read_csv(caminho_csv, encoding_errors= 'ignore', sep= ";", decimal = ",")
     dataframe = df.iloc[:, [0,11,17,18,19,23]] #Selecionei apenas algumas colunas por conta da memoria do PC. No futuro pretendo explorar melhor as outras features :)
     return dataframe

#Webscraping dos documentos:

def extract ():
    #Habilitando todas as abas importantes:

    #Define url
    url = "http://geosampa.prefeitura.sp.gov.br/PaginasPublicas/_SBC.aspx"

    #Setup the driver
    driver = webdriver.Chrome(ChromeDriverManager().install())  

    #Open the webpage
    driver.get(url)

    time.sleep(5)

    #Open Javascript objeto 'Dados Abertos'
    mapa_element = driver.execute_script('return document.getElementById("MapaCabecalhoR").children[0];')
    driver.execute_script("arguments[0].click();", mapa_element)

    driver.implicitly_wait(5)

    #Selecionar "Cadastro" from "cboCamadas"
    select_cbo = Select(driver.find_element_by_id('cboCamadas'))
    select_cbo.select_by_visible_text('Cadastro')

    driver.implicitly_wait(5)

    #Selecionar "IPTU" from "cboSubCamadas"
    select_subcbo = Select(driver.find_element_by_id('cboSubCamadas'))
    select_subcbo.select_by_visible_text('IPTU')

    #Selecionar "XLS_CSV"
    driver.find_element(By.XPATH, value = '//*[contains(text(), "XLS_CSV")]').click()

    time.sleep(20)

    #Iniciando processo de download dos documentos de diretorio remoto para local

    n_downloads = 0
    n_documentos = contagem_documentos(driver)
    ano_iptu = 1994 #Primeiro ano no arquivo é 1995

    while True:
        if n_documentos == 0:
            break
        else:
            n_documentos -= 1
            n_downloads += 1 
            ano_iptu += 1
            if n_downloads%8 != 0:
                download_iptu(ano_iptu,driver)
                time.sleep(200)
            else:
                download_iptu(ano_iptu,driver)
                time.sleep(200)
                parafrente_pagina(driver)
                time.sleep(50)

    #Extração dos dados arquivos .zip brutos obtidos em website em objetos programáveis (Dataframes)

    #Armazenar os caminhos dos arquivos .zip baixados em uma lista:
    n_documentos = contagem_documentos(driver)
    ano_iptu = 1995
    lista_caminhos = []

    for n_caminho in range(0, n_documentos):
        lista_caminhos.append(caminho_iptu(ano_iptu))
        ano_iptu += 1

    #Extraia todos os zipfiles:
    for caminho in lista_caminhos:
         extrair_zipfile(caminho)

    #Defina um dataframe vazio para armazenar os dados extraídos dos zip files:
    dados_extraidos = pd.DataFrame(columns=["contribuinte_n","CEP","terreno_valorm2","construcao_valorm2","imovel_idade","propriedade_tipo", "iptu_ano"])
    contador = 0
    ano_iptu = 1995
    
    #Adicione todos os dados dos arquivos .csv para o DataFrame definido acima:
    for csv in glob.glob("./csv/*.csv"):
        if contador%2 == 0:
            df = extrair_csv(csv)
            df["iptu_ano"] = ano_iptu
            df.columns = ["contribuinte_n","CEP","terreno_valorm2","construcao_valorm2","imovel_idade","propriedade_tipo", "iptu_ano"]
            dados_extraidos = pd.concat([dados_extraidos, df], ignore_index= True)
        contador += 1
        ano_iptu += 1

    return dados_extraidos

### 4.4 Transform - Uniformização dos dados para análise

In [4]:
def transform(data):
        #The dataset is clean and tidy. Therefore, it will not need too much change
        
        #Deletar linhas com NA em mais do que 6 colunas, pois isso não trará nenhum ganho de informação em um dataset de 7 colunas
        data = data[data.isnull().sum(axis=1) < 6]
        
        return data

### 4.5 Load - Transformação de objeto programável (DataFrame) para banco de dados .csv

In [5]:
def load(targetfile, data_to_load):
    data_to_load.to_csv(targetfile, index=False)

### 4.6 Função para acompanhamento da performance de cada etapa do algoritmo ETL

In [6]:
from datetime import datetime
def log(message):
    timestamp_format = '%Y-%h-%d-%H:%M:%S' # Ano-NomedoMes-Dia-Hora-Minuto-Segundo
    now = datetime.now() # Obtenha a data e o tempo preciso do processo
    timestamp = now.strftime(timestamp_format)
    with open("logfile.txt","a") as f:
        f.write(timestamp + ',' + message + '\n')

### 4.7 Aplicando a metodologia de extração de dados ETL

In [7]:
log("ETL Job Started")

log("Extract phase Started")
dados_iptus = extract()
log("Extract phase Ended")

log("Transform phase Started")
transformed_data = transform(dados_iptus)
log("Transform phase Ended")

log("Load phase Started")
load(targetfile,transformed_data)
log("Load phase Ended")

log("ETL Job Ended")